In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


In [2]:
training_data = datasets.FashionMNIST(
        root = 'data',
        download=True,
        transform=ToTensor(),
        train = True
        
    )
test_data = datasets.FashionMNIST(
    root = 'data',
    download=True,
    train = False,
    transform = ToTensor()
)

In [3]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size= batch_size)
test_dataloader = DataLoader(test_data,batch_size = batch_size)

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr =1e-3)

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.306706  [    0/60000]
loss: 2.288608  [ 6400/60000]
loss: 2.268408  [12800/60000]
loss: 2.263254  [19200/60000]
loss: 2.245265  [25600/60000]
loss: 2.218421  [32000/60000]
loss: 2.239785  [38400/60000]
loss: 2.202091  [44800/60000]
loss: 2.193430  [51200/60000]
loss: 2.167691  [57600/60000]
Test Error: 
 Accuracy: 40.6%, Avg loss: 2.159378 

Epoch 2
-------------------------------
loss: 2.169281  [    0/60000]
loss: 2.157790  [ 6400/60000]
loss: 2.101142  [12800/60000]
loss: 2.116626  [19200/60000]
loss: 2.077385  [25600/60000]
loss: 2.012526  [32000/60000]
loss: 2.052503  [38400/60000]
loss: 1.969535  [44800/60000]
loss: 1.969471  [51200/60000]
loss: 1.910438  [57600/60000]
Test Error: 
 Accuracy: 54.8%, Avg loss: 1.901484 

Epoch 3
-------------------------------
loss: 1.930239  [    0/60000]
loss: 1.902039  [ 6400/60000]
loss: 1.785066  [12800/60000]
loss: 1.825696  [19200/60000]
loss: 1.735587  [25600/60000]
loss: 1.670556  [32000/600

In [ ]:
torch.save(model.state_dict(),'model.pth')
print("Saved PyTorch Model State to model.pth")

In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load('model.pth'))

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

In [ ]:
model.eval()
x,y = test_data[0][0], test_data[0][1]

with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')